In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.inspection import permutation_importance
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix



In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0].astype(int)
df_grouped_meal = df_meal.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
#df_grouped_meal['s_id'] = df_meal.groupby('group')['s_id'].first()
df_grouped_meal.index.name = 's_id'

# Reset the index
df_play = df_play.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0].astype(int)
df_grouped_play = df_play.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'
df_grouped_play.index.name = 's_id'


# Reset the index of the grouped dataframes
#df_grouped_meal = df_grouped_meal.reset_index()
#df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'



In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
#df_grouped_meal = df_grouped_meal.reset_index(level=0, drop=True)
meal_data = pd.concat([df_grouped_meal, speech_meal], axis=1)
meal_data = meal_data.dropna()

In [ ]:
play_data = pd.concat([df_grouped_play, speech_play], axis=1)
#meal_data = meal_data.drop(columns=['index'])
play_data = play_data.dropna()

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
speech.set_index('group', append=True, inplace=True)
speech['label'] = speech['label'].map({'no_rad': 0, 'rad': 1})


In [ ]:
# select only meal group from speech index
speech_meal = speech.xs('meal', level='group')
speech_play = speech.xs('play', level='group')

## Meal multimodal model first

In [ ]:

X = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = meal_data['label'].iloc[:, 0]   


remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)

classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear', probability=True), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
conf_matrix_sum = np.zeros((2, 2))
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': grid_search.best_params_.get('preprocessor__pca_pipeline__pca__n_components', None)
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }
    best_model_info = max(best_models.values(), key=lambda x: x['best_score'])
    best_model = best_model_info['best_estimator']

    # Use the best model to make predictions on the test set
    y_pred = best_model.predict(X_shuffled)

    # Compute the confusion matrix for this iteration
    conf_matrix = confusion_matrix(y_shuffled, y_pred)
    conf_matrix_sum += conf_matrix

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
average_conf_matrix = conf_matrix_sum / 10

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
pipeline_meal = best_models['xgb']['best_estimator']

In [ ]:
pipeline_meal

## Play speech child model

In [ ]:


# Perform a grid search for each classifier
X = speech_play[['Proportion speech child']]
y = speech_play['label']
groups = speech_meal.index

# Create a GroupKFold object
gkf = KFold(n_splits=5)
best_models = {}
# Define the classifiers and their parameters
classifiers = [
('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf', probability=True), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y,  random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro',n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
            }
# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()


In [ ]:
pipeline_play = best_models['svc_rbf']['best_estimator']

In [ ]:

X = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = play_data['label'].iloc[:, 0]   


remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)

classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear', probability=True), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf', probability=True), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
conf_matrix_sum = np.zeros((2, 2))
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': grid_search.best_params_.get('preprocessor__pca_pipeline__pca__n_components', None)
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }
    best_model_info = max(best_models.values(), key=lambda x: x['best_score'])
    best_model = best_model_info['best_estimator']

    # Use the best model to make predictions on the test set
    y_pred = best_model.predict(X_shuffled)

    # Compute the confusion matrix for this iteration
    conf_matrix = confusion_matrix(y_shuffled, y_pred)
    conf_matrix_sum += conf_matrix

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
average_conf_matrix = conf_matrix_sum / 10

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
pipeline_play = best_models['svc_linear']['best_estimator']

# Intersect the datasets

In [ ]:
meal_data_selected = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group','label']]
play_data_selected = play_data[['Proportion speech child', 'group', 'label']]

# Find the common indices
common_indices = meal_data_selected.index.intersection(play_data_selected.index)

# Filter the DataFrames to keep only the common indices
meal_data_filtered = meal_data_selected.loc[common_indices]
play_data_filtered = play_data_selected.loc[common_indices]

# Concatenate the filtered DataFrames
combined_data = pd.concat([meal_data_filtered, play_data_filtered], axis=1)
combined_labels = meal_data_filtered['label'].iloc[:,0]

In [ ]:
X_meal = meal_data_filtered.drop(columns=['label', 'group'])
X_play = play_data_filtered.drop(columns=['label', 'group'])
y_meal = meal_data_filtered['label'].iloc[:,0]
y_play = play_data_filtered['label'].iloc[:,0]  

In [ ]:
X_meal = X_meal.add_suffix('_meal')
X_play = X_play.add_suffix('_play')

In [ ]:
X_combined = pd.concat([X_meal, X_play], axis=1)
X_combined.columns

In [ ]:
X_combined.columns

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

subset_pca = ['cg_movement_mean_meal',
       'cg_movement_var_meal', 'cg_movement_min_meal', 'cg_movement_max_meal',
       'child_movement_mean_meal', 'child_movement_var_meal',
       'child_movement_min_meal', 'child_movement_max_meal']
remaining_features = [feat for feat in X_combined.columns if feat not in subset_pca]

preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Define the meta-model
meta_models = [
    ('svc', Pipeline([('preprocessor', preprocessor), ('svc', SVC(probability=True))])),
    ('logistic_regression', Pipeline([('preprocessor', preprocessor), ('logistic_regression', LogisticRegression())])),
    ('random_forest', Pipeline([('preprocessor', preprocessor), ('random_forest', RandomForestClassifier())]))
]
param_grids = {
    'svc': {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc__C': [0.01, 0.1, 1, 10, 100],
        'svc__gamma': [0.01, 0.1, 1, 10, 100],
        'svc__kernel': ['linear', 'rbf']
    },
    'logistic_regression': {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'logistic_regression__C': [0.01, 0.1, 1, 10, 100],
        'logistic_regression__penalty': ['l1', 'l2'],
        'logistic_regression__solver': ['liblinear', 'saga']
    },
    'random_forest': {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'random_forest__n_estimators': [10, 50, 100, 200],
        'random_forest__max_depth': [None, 5, 10, 15],
        'random_forest__min_samples_split': [2, 5, 10]
    }
}

results = []
# Define the meta-model




for i in range(10):
    # Shuffle the data with a different random state each time
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    X_meal = meal_data_filtered.drop(columns=['label', 'group'])
    X_play = play_data_filtered.drop(columns=['label', 'group'])
    meal_predictions = cross_val_predict(pipeline_meal, X_meal, y_meal, cv=cv, method='predict')
    play_predictions = cross_val_predict(pipeline_play, X_play, y_play, cv=cv, method='predict')


    meal_predictions_df = pd.DataFrame(meal_predictions, columns=['predicted_class_meal'], index=X_meal.index)
    play_predictions_df = pd.DataFrame(meal_predictions, columns=['predicted_class_play'], index=X_meal.index)
    combined_predictions = pd.concat([meal_predictions_df, play_predictions_df], axis=1).dropna()
    combined_labels = pd.concat([y_meal, y_play],axis=1).dropna().iloc[:,0]     
    X_meal = X_meal.add_suffix('_meal')
    X_play = X_play.add_suffix('_play')
    X_combined = pd.concat([X_meal, X_play], axis=1)
    X_combined = pd.concat([X_combined, combined_predictions], axis=1)
    combined_groups = X_combined.index

    X_shuffled, y_shuffled, groups_shuffled = shuffle(X_combined, combined_labels, combined_groups, random_state=i)

    
    cv = GroupKFold(n_splits=5)
    for model_name, meta_model in meta_models:
        grid_search = GridSearchCV(meta_model, param_grids[model_name], cv=cv, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)
        best_meta_model = grid_search.best_estimator_
        
        accuracy_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='accuracy', n_jobs=-1)
        precision_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='precision_weighted', n_jobs=-1)
        recall_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='recall_weighted', n_jobs=-1)
        f1_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='f1_weighted', n_jobs=-1)
        
        results.append({
            'random_state': i,
            'model': model_name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'f1_score': f1_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
X_meal

In [ ]:
results_df.groupby('model').mean()

In [ ]:
cross_val_predict(pipeline_meal, X_meal, y_meal, cv=cv, method='predict')

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

subset_pca = ['cg_movement_mean', 'cg_movement_var',
       'cg_movement_min', 'cg_movement_max', 'child_movement_mean',
       'child_movement_var', 'child_movement_min', 'child_movement_max']
remaining_features = [feat for feat in X_combined.columns if feat not in subset_pca]

preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Define the meta-model
meta_models = [
    ('svc', Pipeline([('preprocessor', preprocessor), ('svc', SVC(probability=True))])),
    ('logistic_regression', Pipeline([('preprocessor', preprocessor), ('logistic_regression', LogisticRegression())])),
    ('random_forest', Pipeline([('preprocessor', preprocessor), ('random_forest', RandomForestClassifier())]))
]
param_grids = {
    'svc': {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc__C': [0.01, 0.1, 1, 10, 100],
        'svc__gamma': [0.01, 0.1, 1, 10, 100],
        'svc__kernel': ['linear', 'rbf']
    },
    'logistic_regression': {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'logistic_regression__C': [0.01, 0.1, 1, 10, 100],
        'logistic_regression__penalty': ['l1', 'l2'],
        'logistic_regression__solver': ['liblinear', 'saga']
    },
    'random_forest': {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'random_forest__n_estimators': [10, 50, 100, 200],
        'random_forest__max_depth': [None, 5, 10, 15],
        'random_forest__min_samples_split': [2, 5, 10]
    }
}

results = []
# Define the meta-model




for i in range(10):
    # Shuffle the data with a different random state each time
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    meal_predictions = cross_val_predict(pipeline_meal, X_meal, y_meal, cv=cv, method='predict')
    play_predictions = cross_val_predict(pipeline_play, X_play, y_play, cv=cv, method='predict')


    meal_predictions_df = pd.DataFrame(meal_predictions,columns=['predicted_class'], index=X_meal.index)
    meal_predictions_df['group'] = 'meal'
    
    play_predictions_df = pd.DataFrame(play_predictions,columns=['predicted_class'], index=X_play.index)
    play_predictions_df['group'] = 'play'
    
    combined_predictions = pd.concat([meal_predictions_df, play_predictions_df], axis=0).dropna()

    combined_labels = pd.concat([y_meal, y_play],axis=0).dropna()

    X_combined = pd.concat([X_meal, X_play], axis=0)
    X_combined = pd.concat([X_combined, combined_predictions], axis=1)
    X_combined = X_combined.drop(columns=['group'])
    combined_groups = X_combined.index

    X_shuffled, y_shuffled, groups_shuffled = shuffle(X_combined, combined_labels, combined_groups, random_state=i)


    cv = GroupKFold(n_splits=5)
    for model_name, meta_model in meta_models:
        grid_search = GridSearchCV(meta_model, param_grids[model_name], cv=cv, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)
        best_meta_model = grid_search.best_estimator_
        
        accuracy_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='accuracy', n_jobs=-1)
        precision_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='precision_weighted', n_jobs=-1)
        recall_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='recall_weighted', n_jobs=-1)
        f1_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='f1_weighted', n_jobs=-1)
        
        results.append({
            'random_state': i,
            'model': model_name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'f1_score': f1_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
results_df.groupby('model').mean()